In [1]:
import os
# os.chdir(r"/content/drive/MyDrive/billing_features/raw/")
import math
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import missingno as msno
import pickle
import lightgbm
import xgboost as xgb
#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score,average_precision_score
from sklearn.metrics import precision_recall_fscore_support 
from sklearn.metrics import roc_curve,precision_recall_curve
from sklearn.metrics import auc as auc_score
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import utils

In [15]:
data_dir="/app/models/dij22"
df_buffer_0_hist_3_pickle=pd.read_pickle(os.path.join(data_dir,"df_buffer_3_hist_3_pickle"))
df_buffer_3_hist_3_pickle=pd.read_pickle(os.path.join(data_dir,"df_buffer_3_hist_3_pickle"))

In [3]:
def train_test_data(data,feature_type,test_yr):
    df=data.copy()
    all_var=df.columns.tolist()
    exclude_cols=['policy_id', 'pivot_date', 'churn',"year","month","orig_policy_eff_dt", "policy_anniv_dt", "policy_term_dt"]
    exclude_var=[]
    
    if feature_type=="original":
        for col in all_var:
            if col[:2] in ["L1","L2","L3","L6","L12",'d1','d2','d3','d6','d12',"r1","r2","r3","r6","r12"]:
                exclude_var.append(col)
                
    elif feature_type=="original+rolling window":
        for col in all_var:
            if col[:2] in ['d1','d2','d3','d6','d12',"r1","r2","r3","r6","r12"]:
                exclude_var.append(col)
    
    elif feature_type=="original+rolling window+delta":
        for col in all_var:
            if col[:2] in ["r1","r2","r3","r6","r12"]:
                exclude_var.append(col)
                
    elif feature_type=="original+rolling window+delta+ratio":
        exclude_var=[]
    
    else:
        raise NotImplemented("Unknown feature type.")
                
    df.drop(exclude_var, axis=1,inplace=True)
    train_data=df[df["year"]!=test_yr]
    test_data=df[df["year"]==test_yr]

    y_train=train_data.loc[:,"churn"]
    y_test=test_data.loc[:,"churn"]
    X_train=train_data.drop(exclude_cols, axis=1)
    X_test=test_data.drop(exclude_cols, axis=1)

    return X_train, X_test, y_train, y_test


def model_eval(X_train,X_test,y_train,y_test):
    opt_params = utils.bayes_parameter_opt_lgb(X_train, y_train, init_round=5, opt_round=10, n_folds=3, random_seed=6,n_estimators=10000)
    train_data = lightgbm.Dataset(X_train, label=y_train)
    test_data = lightgbm.Dataset(X_test, label=y_test)
    model = lightgbm.train(opt_params,
                           train_data,
                           valid_sets=[train_data,test_data],
                           num_boost_round=5000,
                           early_stopping_rounds=100)

    feature_importance= (pd.DataFrame({
        'feature': model.feature_name(),
        'importance': model.feature_importance(),
        }).sort_values('importance', ascending=False))
    feature_importance["rank"]=list(range(len(model.feature_name())))
    feature_importance=feature_importance.loc[:,["rank","feature","importance"]].reset_index(drop=True)
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    train_eval=utils.model_evaluate(y_train, train_preds)
    test_eval=utils.model_evaluate(y_test, test_preds)
    
    return model, feature_importance, train_eval, test_eval

def evaluation_table(eval_v1,eval_v2,eval_v3,eval_v4,type):
    dict_data={}
    dict_data["Features"]=["original feature","original + rolling window feature","original + rolling window + delta feature","original + rolling window + delta  + ratio feature"]
    # dict_data["# of feature"]=[len(feat_1),len(feat_2),len(feat_3)] 
    dict_data["# of sample"]=[eval_v1['nb_example'],eval_v2['nb_example'],eval_v3['nb_example'],eval_v4['nb_example']]
    # dict_data["true_prediction"]=[eval_v1['true_prediction'],eval_v2['true_prediction'],eval_v3['true_prediction']]
    # dict_data["false_prediction"]=[eval_v1['false_prediction'],eval_v2['false_prediction'],eval_v3['false_prediction']]
    # dict_data["accuracy"]=[eval_v1['accuracy'],eval_v2['accuracy'],eval_v3['accuracy']]
    dict_data["precision"]=[eval_v1['precision'],eval_v2['precision'],eval_v3['precision'],eval_v4['precision']]  
    dict_data["recall"]=[eval_v1['recall'],eval_v2['recall'],eval_v3['recall'],eval_v4['recall']] 
    dict_data["f1_score"]=[eval_v1['f1_score'],eval_v2['f1_score'],eval_v3['f1_score'],eval_v4['f1_score']] 
    dict_data["ROC-AUC"]=[eval_v1['AUC'],eval_v2['AUC'],eval_v3['AUC'],eval_v4['AUC']] 
    dict_data["pr-auc"]=[eval_v1['pr_auc'],eval_v2['pr_auc'],eval_v3['pr_auc'],eval_v4['pr_auc']] 
    data_df=pd.DataFrame(dict_data)
    # data_df=data_df.set_index("Model Type")
    # data_df.style.format({"# of sample":"{:,}","true_prediction":"{:,}","false_prediction":"{:,}","accuracy":"{:.2%}","precision":"{:.2%}","recall":"{:.2%}","f1_score":"{:.2%}","ROC-AUC":"{:.2%}","pr-auc":"{:.2%}"})\
    return data_df.style.format({"# of sample":"{:,}","precision":"{:.2%}","recall":"{:.2%}","f1_score":"{:.2%}","ROC-AUC":"{:.2%}","pr-auc":"{:.2%}"})\
    .set_caption(f"Model Performance Comparison {type}")\
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'red'),
            ('font-size', '20px')
        ]
    }])

# 3 month buffer, 3 month fixed-window

In [4]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_3_pickle,feature_type="original",test_yr=2022)
print("{:<30}{:<20,}".format('training features: ', len(X_train)))
print("{:<30}{:<20,}".format('testing features: ', len(X_test)))

pd.DataFrame(y_test, columns=["churn"])["churn"].value_counts(dropna=False,normalize=True).to_frame().style.format({"churn":"{:.2%}"})

training features:            822,696             
testing features:             49,307              


,churn
0,97.72%
1,2.28%


In [6]:
model_v00, feature_importance_v00, train_eval_v00, test_eval_v00=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6635   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6241   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6579   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.6703   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6385   |  0.9864   |  0.3264   |  0.6526   |  38.59    |  9.692    |  47.71    |  66.6     |  52.98    |  0.8559   |
|  6        |  0.6632   |  0.6952   |  0.7717   |  0.3244   | 

In [7]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_3_pickle,feature_type="original+rolling window",test_yr=2022)
model_v10, feature_importance_v10, train_eval_v10, test_eval_v10=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.6771   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6366   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6686   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.69     |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6579   |  0.9864 

In [8]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_3_pickle,feature_type="original+rolling window+delta",test_yr=2022)
model_v20, feature_importance_v20, train_eval_v20, test_eval_v20=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.6983   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6538   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6821   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.71     |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6728   |  0.9864 

In [9]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_3_pickle,feature_type="original+rolling window+delta+ratio",test_yr=2022)
model_v30, feature_importance_v30, train_eval_v30, test_eval_v30=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.6963   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6633   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6855   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.7142   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6741   |  0.9864 

In [10]:
evaluation_table(train_eval_v00,train_eval_v10,train_eval_v20,train_eval_v30,"Training Set")

,Features,# of sample,precision,recall,f1_score,ROC-AUC,pr-auc
0,original feature,"822,696",13.03%,28.79%,17.94%,72.97%,11.16%
1,original + rolling window feature,"822,696",20.32%,35.80%,25.93%,81.96%,19.12%
2,original + rolling window + delta feature,"822,696",34.74%,49.39%,40.79%,91.06%,38.66%
3,original + rolling window + delta + ratio feature,"822,696",12.84%,35.88%,18.91%,73.75%,11.36%


In [11]:
evaluation_table(test_eval_v00,test_eval_v10,test_eval_v20,test_eval_v30,"Test Set")

,Features,# of sample,precision,recall,f1_score,ROC-AUC,pr-auc
0,original feature,"49,307",8.44%,22.65%,12.30%,70.09%,6.13%
1,original + rolling window feature,"49,307",8.35%,24.07%,12.39%,70.60%,6.47%
2,original + rolling window + delta feature,"49,307",4.77%,18.65%,7.59%,63.53%,3.68%
3,original + rolling window + delta + ratio feature,"49,307",5.51%,30.91%,9.36%,68.52%,4.01%


In [12]:
def feature_importance(model):
    df_feature_importance = (
        pd.DataFrame({
            'feature': model.feature_name(),
            'importance': model.feature_importance(),
        }).sort_values('importance', ascending=False)
    )
    df_feature_importance["rank"]=list(range(len(model.feature_name())))
    df_feature_importance=df_feature_importance.loc[:,["rank","feature","importance"]].reset_index(drop=True)
    return df_feature_importance

df_feature_importance_v0=feature_importance(model_v00)
df_feature_importance_v1=feature_importance(model_v10)
df_feature_importance_v2=feature_importance(model_v20)
df_feature_importance_v3=feature_importance(model_v30)
f0=df_feature_importance_v0.loc[:30,['rank','feature']].rename(columns={"feature":"original feature"})
f1=df_feature_importance_v1.loc[:30,['rank','feature']].rename(columns={"feature":"original + rolling window feature"})
f2=df_feature_importance_v2.loc[:30,['rank','feature']].rename(columns={"feature":"original + rolling window + delta feature"})
f3=df_feature_importance_v3.loc[:30,['rank','feature']].rename(columns={"feature":"original + rolling window + delta + ratio feature"})

feature_importance=pd.merge(f0,f1,how="inner",on="rank")
feature_importance=pd.merge(feature_importance,f2,how="inner",on="rank")
feature_importance=pd.merge(feature_importance,f3,how="inner",on="rank")
# feature_importance.style.format().set_caption("Top 20 important Features").set_table_styles([{
#     'selector': 'caption',
#     'props': [
#         ('color', 'red'),
#         ('font-size', '20px')
#     ]
# }])
feature_importance

,rank,original feature,original + rolling window feature,original + rolling window + delta feature,original + rolling window + delta + ratio feature
0,0,survival_month,survival_month,survival_month,survival_month
1,1,PaidBillDueDays,L12_AvgPdBillLstGenDays,d12_AvgPdBillLstGenDays,r12_Lag12_cntPaidFull
2,2,AvgPdBilldueDays,L12_PaidBillLastGenDays,d12_AvgPdBilldueDays,r6_Lag12_cntBills
3,3,Lag12_cntPaidFull,L3_AvgPdBillLstGenDays,d2_AvgPdBillLstGenDays,r12_Lag12_cntBills
4,4,AvgPdBillLstGenDays,AvgPdBillLstGenDays,L12_AvgPdBillLstGenDays,L12_AvgPaidFullCnt
5,5,PaidBillLastGenDays,L6_AvgPdBillLstGenDays,d6_PaidBillLastGenDays,L12_AvgBillGenCnt
6,6,CurrPaidAmt,L12_AvgPdBilldueDays,d12_PaidBillDueDays,r6_Lag12_cntPaidFull
7,7,Lag12_cntBills,L2_AvgPdBillLstGenDays,d6_AvgPdBilldueDays,L12_AvgPdBillLstGenDays
8,8,OrigBillAmt,L6_AvgPdBilldueDays,d12_PaidBillLastGenDays,r12_CurrPaidAmt
9,9,Lag12_cntBillGens,AvgPdBilldueDays,d1_AvgPdBilldueDays,r12_Lag12_cntFirstGenPaidFull


# 0 month buffer, 3 month fixed-window

In [16]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_0_hist_3_pickle,feature_type="original",test_yr=2022)
print("{:<30}{:<20,}".format('training features: ', len(X_train)))
print("{:<30}{:<20,}".format('testing features: ', len(X_test)))

pd.DataFrame(y_test, columns=["churn"])["churn"].value_counts(dropna=False,normalize=True).to_frame().style.format({"churn":"{:.2%}"})

training features:            822,696             
testing features:             49,307              


,churn
0,97.72%
1,2.28%


In [12]:
model_v0, feature_importance_v0, train_eval_v0, test_eval_v0=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.6609   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.64     |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6568   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.6683   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6377   |  0.9864 

In [13]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_6_pickle,feature_type="original+rolling window",test_yr=2022)
model_v1, feature_importance_v1, train_eval_v1, test_eval_v1=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.6865   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6519   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.6779   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.6968   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6646   |  0.9864 

In [14]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_6_pickle,feature_type="original+rolling window+delta",test_yr=2022)
model_v2, feature_importance_v2, train_eval_v2, test_eval_v2=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.7235   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6694   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.7131   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.7358   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6957   |  0.9864 

In [15]:
X_train, X_test, y_train, y_test=train_test_data(df_buffer_3_hist_6_pickle,feature_type="original+rolling window+delta+ratio",test_yr=2022)
model_v3, feature_importance_v3, train_eval_v3, test_eval_v3=model_eval(X_train,X_test,y_train,y_test)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9738161284258351, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9738161284258351
|  1        |  0.7202   |  0.9738   |  0.2812   |  0.5985   |  49.98    |  24.1     |  10.26    |  35.74    |  74.94    |  0.4615   |
|  2        |  0.6757   |  0.9909   |  0.7939   |  0.9862   |  84.63    |  12.59    |  86.15    |  12.12    |  67.99    |  0.258    |
|  3        |  0.7143   |  0.548    |  0.8548   |  0.8278   |  56.28    |  26.84    |  62.05    |  45.01    |  62.09    |  0.4252   |
|  4        |  0.7407   |  0.8202   |  0.5869   |  0.1144   |  87.62    |  23.97    |  71.17    |  32.93    |  25.48    |  0.8056   |
|  5        |  0.6981   |  0.9864 

In [16]:
evaluation_table(train_eval_v0,train_eval_v1,train_eval_v2,train_eval_v3,"Training Set")

,Features,# of sample,precision,recall,f1_score,ROC-AUC,pr-auc
0,original feature,"1,605,389",6.60%,19.27%,9.83%,67.19%,4.85%
1,original + rolling window feature,"1,605,389",11.85%,21.34%,15.24%,75.93%,8.82%
2,original + rolling window + delta feature,"1,605,389",12.94%,20.52%,15.87%,73.89%,8.29%
3,original + rolling window + delta + ratio feature,"1,605,389",15.73%,22.83%,18.63%,76.01%,11.45%


In [17]:
evaluation_table(test_eval_v0,test_eval_v1,test_eval_v2,test_eval_v3,"Test Set")

,Features,# of sample,precision,recall,f1_score,ROC-AUC,pr-auc
0,original feature,"30,619",5.89%,39.43%,10.24%,58.97%,4.75%
1,original + rolling window feature,"30,619",5.34%,44.05%,9.53%,59.65%,5.18%
2,original + rolling window + delta feature,"30,619",5.62%,35.97%,9.72%,59.30%,5.42%
3,original + rolling window + delta + ratio feature,"30,619",41.77%,11.72%,18.31%,67.30%,12.92%
